<a href="https://colab.research.google.com/github/sonalmajumder/rentwiseAI/blob/main/rentwiseAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install transformers datasets torch


In [14]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
import torch


In [15]:
# Load sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

# Example customer review
review = "The car was clean and the pickup process was very smooth!"

# Get sentiment
result = sentiment_pipeline(review)
print("Sentiment:", result)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Sentiment: [{'label': 'POSITIVE', 'score': 0.9976062774658203}]


In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load GPT2 model
generator_tokenizer = AutoTokenizer.from_pretrained("gpt2")
generator_model = AutoModelForCausalLM.from_pretrained("gpt2")

# Generate a response
input_text = "Customer said: The car was clean and the pickup process was very smooth! Response:"
input_ids = generator_tokenizer.encode(input_text, return_tensors="pt")

# Generate text
output = generator_model.generate(input_ids, max_length=50, num_return_sequences=1, pad_token_id=generator_tokenizer.eos_token_id)
generated_text = generator_tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Reply:")
print(generated_text)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Reply:
Customer said: The car was clean and the pickup process was very smooth! Response: I have a car that is very clean and I have been driving it for about 3 years. I have a lot of questions about the car and I am not sure


In [18]:
def analyze_review(review):
    sentiment = sentiment_pipeline(review)[0]['label']
    input_text = f"Customer said: {review} Response:"
    input_ids = generator_tokenizer.encode(input_text, return_tensors="pt")
    output = generator_model.generate(input_ids, max_length=50, num_return_sequences=1, pad_token_id=generator_tokenizer.eos_token_id)
    reply = generator_tokenizer.decode(output[0], skip_special_tokens=True)
    return sentiment, reply


In [19]:
user_review = input("Enter a customer review: ")
sentiment, reply = analyze_review(user_review)

print("\nSentiment:", sentiment)
print("Generated Response:", reply)


Enter a customer review: okay

Sentiment: POSITIVE
Generated Response: Customer said: okay Response: I'm sorry for the delay. I'm sorry for the delay. I'm sorry for the delay. I'm sorry for the delay. I'm sorry for the delay. I'm sorry for the delay. I'm


In [20]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

# Load CSV into pandas
for fn in uploaded.keys():
    df = pd.read_csv(fn)

df.head()


Saving car_rental_train.csv to car_rental_train (1).csv
Saving car_rental_test.csv to car_rental_test (1).csv


,ID,Gender,Age,Customer Status,Customer Service,Satisfaction,Business Area
0,1,Female,68,Active,Staff at pickup were helpful,0,Pickup Process
1,2,Male,40,Inactive,I received an old model with scratches,0,Vehicle Quality
2,3,Male,33,Inactive,Confusing dropoff instructions,0,Dropoff Process
3,4,Male,44,Active,Dropoff was fast and efficient,1,Dropoff Process
4,5,Male,61,Inactive,Difficult to navigate the app,1,App Usability


In [21]:
def analyze_reviews(df):
    sentiments = []
    replies = []
    for review in df['review']:
        sentiment = sentiment_pipeline(review)[0]['label']
        input_text = f"Customer said: {review} Response:"
        input_ids = generator_tokenizer.encode(input_text, return_tensors="pt")
        output = generator_model.generate(input_ids, max_length=50, pad_token_id=generator_tokenizer.eos_token_id)
        reply = generator_tokenizer.decode(output[0], skip_special_tokens=True)
        sentiments.append(sentiment)
        replies.append(reply)
    df['Sentiment'] = sentiments
    df['AI_Response'] = replies
    return df

results_df = analyze_reviews(df)
results_df.head()


KeyError: 'review'

In [22]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import torch
from google.colab import files
import pandas as pd

# Load sentiment analysis pipeline (assuming this is already done in a previous cell)
# sentiment_pipeline = pipeline("sentiment-analysis")

# Load GPT2 model (assuming this is already done in a previous cell)
# generator_tokenizer = AutoTokenizer.from_pretrained("gpt2")
# generator_model = AutoModelForCausalLM.from_pretrained("gpt2")

# %%
# If you haven't already uploaded the file in a previous cell, uncomment and run the following:
# uploaded = files.upload()
#
# # Load CSV into pandas
# for fn in uploaded.keys():
#     df = pd.read_csv(fn)
#
# df.head()


def analyze_reviews(df):
    sentiments = []
    replies = []
    # **CHANGE:** Replace 'review' with the actual column name in your CSV file
    # For example, if the column is named 'customer_review', change df['review'] to df['customer_review']
    review_column_name = 'review' # Change this to the correct column name if it's different
    if review_column_name not in df.columns:
        print(f"Error: Column '{review_column_name}' not found in the DataFrame.")
        print("Available columns are:", df.columns.tolist())
        return None # Or handle the error as appropriate

    for review in df[review_column_name]:
        # Ensure the review is a string before passing it to the sentiment pipeline
        if isinstance(review, str):
            sentiment = sentiment_pipeline(review)[0]['label']
            input_text = f"Customer said: {review} Response:"
            input_ids = generator_tokenizer.encode(input_text, return_tensors="pt")
            output = generator_model.generate(input_ids, max_length=50, pad_token_id=generator_tokenizer.eos_token_id)
            reply = generator_tokenizer.decode(output[0], skip_special_tokens=True)
            sentiments.append(sentiment)
            replies.append(reply)
        else:
            # Handle non-string values if necessary, e.g., append None or a placeholder
            sentiments.append(None)
            replies.append(None)
            print(f"Warning: Skipping non-string review: {review}")

    df['Sentiment'] = sentiments
    df['AI_Response'] = replies
    return df

# Assuming 'df' has been loaded from the CSV in a previous cell
# Call the function with the loaded DataFrame
results_df = analyze_reviews(df)

# Check if results_df is not None before trying to display it
if results_df is not None:
    results_df.head()

Error: Column 'review' not found in the DataFrame.
Available columns are: ['ID', 'Gender', 'Age', 'Customer Status', 'Customer Service', 'Satisfaction', 'Business Area']


In [23]:
!pip install gradio

import gradio as gr

def process_input(review):
    sentiment = sentiment_pipeline(review)[0]['label']
    input_text = f"Customer said: {review} Response:"
    input_ids = generator_tokenizer.encode(input_text, return_tensors="pt")
    output = generator_model.generate(input_ids, max_length=50, pad_token_id=generator_tokenizer.eos_token_id)
    reply = generator_tokenizer.decode(output[0], skip_special_tokens=True)
    return sentiment, reply

interface = gr.Interface(fn=process_input,
                         inputs="text",
                         outputs=["text", "text"],
                         title="Car Rental Review Sentiment + AI Reply",
                         description="Enter a customer review to see sentiment and generate a smart AI reply.")

interface.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.8 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://942002a5d4f06c76e5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory t

In [24]:
def analyze_review_with_emojis(review):
    sentiment = sentiment_pipeline(review)[0]['label']
    tone_emoji = "😊" if sentiment == "POSITIVE" else "😞"
    input_text = f"Customer said: {review} Response:"
    input_ids = generator_tokenizer.encode(input_text, return_tensors="pt")
    output = generator_model.generate(input_ids, max_length=50, pad_token_id=generator_tokenizer.eos_token_id)
    reply = generator_tokenizer.decode(output[0], skip_special_tokens=True)
    return sentiment, reply + " " + tone_emoji


In [25]:
import matplotlib.pyplot as plt

# After analysis
sentiment_counts = results_df['Sentiment'].value_counts()
sentiment_counts.plot(kind='bar', title='Sentiment Analysis of Reviews')
plt.show()


TypeError: 'NoneType' object is not subscriptable

In [26]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import torch
from google.colab import files
import pandas as pd

# Load sentiment analysis pipeline (assuming this is already done in a previous cell)
# sentiment_pipeline = pipeline("sentiment-analysis")

# Load GPT2 model (assuming this is already done in a previous cell)
# generator_tokenizer = AutoTokenizer.from_pretrained("gpt2")
# generator_model = AutoModelForCausalLM.from_pretrained("gpt2")

# %%
# If you haven't already uploaded the file in a previous cell, uncomment and run the following:
# uploaded = files.upload()
#
# # Load CSV into pandas
# for fn in uploaded.keys():
#     df = pd.read_csv(fn)
#
# df.head()


def analyze_reviews(df):
    sentiments = []
    replies = []
    # **CHANGE:** Replace 'review' with the actual column name in your CSV file
    # For example, if the column is named 'customer_review', change df['review'] to df['customer_review']
    # >>>>>>>>>> CHANGE THIS LINE <<<<<<<<<<
    review_column_name = 'review' # Change this to the correct column name if it's different
    # <<<<<<<<<< END CHANGE <<<<<<<<<<
    if review_column_name not in df.columns:
        print(f"Error: Column '{review_column_name}' not found in the DataFrame.")
        print("Available columns are:", df.columns.tolist())
        return None # Or handle the error as appropriate

    for review in df[review_column_name]:
        # Ensure the review is a string before passing it to the sentiment pipeline
        if isinstance(review, str):
            sentiment = sentiment_pipeline(review)[0]['label']
            input_text = f"Customer said: {review} Response:"
            input_ids = generator_tokenizer.encode(input_text, return_tensors="pt")
            output = generator_model.generate(input_ids, max_length=50, pad_token_id=generator_tokenizer.eos_token_id)
            reply = generator_tokenizer.decode(output[0], skip_special_tokens=True)
            sentiments.append(sentiment)
            replies.append(reply)
        else:
            # Handle non-string values if necessary, e.g., append None or a placeholder
            sentiments.append(None)
            replies.append(None)
            print(f"Warning: Skipping non-string review: {review}")


    df['Sentiment'] = sentiments
    df['AI_Response'] = replies
    return df

# Assuming 'df' has been loaded from the CSV in a previous cell
# Call the function with the loaded DataFrame
results_df = analyze_reviews(df)

# Check if results_df is not None before trying to display it
if results_df is not None:
    results_df.head()

Error: Column 'review' not found in the DataFrame.
Available columns are: ['ID', 'Gender', 'Age', 'Customer Status', 'Customer Service', 'Satisfaction', 'Business Area']
